In [6]:
!pip install --upgrade category_encoders
!pip install xgboost
!pip install hyperopt




[notice] A new release of pip is available: 23.0.1 -> 23.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.1
[notice] To update, run: python -m pip install --upgrade pip
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 24.3 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.9/840.9 kB 20.2 MB/s eta 0:00:0000:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.1/77.1 kB 2.8 MB/s eta 0:00:00
  Created wheel for future: filename=future-0.18.3-py3-none-any.whl size=492022 sha256=27dc428e2fac663b59877c1b7cdd849ef349219a82ce7875c44b54bd8b7874cf
  Stored in directory: /home/codespace/.cache/pip/wheels/5e/a9/47/f118e66afd12240e4662752cc22cefae5d97275623aa8ef57d
Successfully built future

[notice] A new release of pip is available: 23.0.1 -> 23.1
[notice] To update, run: python

In [ ]:
import pandas as pd
import numpy as np
import sklearn as skl
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from sklearn import tree

from sklearn.ensemble import VotingRegressor

from sklearn.model_selection import cross_val_score

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor

from sklearn.ensemble import ExtraTreesRegressor
from sklearn.model_selection import train_test_split


from sklearn.metrics import mean_absolute_percentage_error, mean_squared_log_error, mean_squared_error, mean_absolute_error, r2_score
import category_encoders as ce
from sklearn.pipeline import Pipeline
from numpy import mean
from numpy import std
from sklearn.preprocessing import StandardScaler

from sklearn.preprocessing import power_transform
from sklearn.preprocessing import PowerTransformer

from xgboost import XGBRegressor
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from hyperopt import tpe, STATUS_OK, Trials, hp, fmin, STATUS_OK, space_eval

from sklearn.model_selection import RandomizedSearchCV
import hyperopt
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from sklearn.model_selection import cross_val_score
from hyperopt import hp, fmin, tpe
from hyperopt.pyll.base import scope
import xgboost as xgb
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler

In [7]:
def symmetric_mean_absolute_percentage_error(a, f):
    return 1/len(a) * np.sum(2 * np.abs(f-a) / (np.abs(a) + np.abs(f))*100)


Preprocessing

In [8]:
def remove_outliers(df, target_name):
    y=df[target_name]
    data_mean, data_std = mean(y), std(y)
# identify outliers
    cut_off = data_std * 3
    lower, upper = data_mean - cut_off, data_mean + cut_off
# identify outliers

    df_ = df[df[target_name].between(lower, upper)]
    return df_

def scaler(df):
    ss=StandardScaler()
    df_=ss.fit_transform(df)
    return df_



In [9]:
def create_metrics_tag():
    return {'Specific': [], 'MAPE': [], 'SMAPE':[],  'MSLE': [], 'MSE':[], 'MAE':[], 'R2':[]}

def get_metrics(a, p, name):

    metrics = {'MAPE' : mean_absolute_percentage_error, 'SMAPE': symmetric_mean_absolute_percentage_error, 'MSLE': mean_squared_log_error, 'MSE': mean_squared_error, 'MAE': mean_absolute_error, 'R2':r2_score}
    tags=create_metrics_tag()

    for metric in metrics.keys():
        func=metrics[metric]
        val =func(a, p)
        tags[metric].append(val)

    tags['Specific'].append(name)
    df_ = pd.DataFrame(tags)
    return df_


In [10]:
def handleColumns(dropCols= []):
    CatColumns=['sex', 'smoker', 'region', 'weight', 'senility']
    ValColumns=['bmi',  'multiplier', 'children', 'age']
    catCols = [i for i in CatColumns if i not in dropCols]
    valCols = [i for i in ValColumns if i not in dropCols]
    return catCols, valCols

def getPipe(model, dropCols= []):

    catCols,valCols = handleColumns(dropCols)
    encoder = ce.JamesSteinEncoder(cols=catCols)
    ct = ColumnTransformer([
        ('scaler', StandardScaler(), valCols+ catCols)
    ], remainder='passthrough') 

    modelpipe = Pipeline([
  ('encode_categorical', encoder),
  ('transformer', ct),
  ('classifier', model)
  ])

    return modelpipe

In [11]:
def preprocess(df,scal= False, dropN= None ):

    df_=df.copy()
    if(dropN):
        df_=df_.drop(dropN, axis=1)
    df_['weight']= pd.cut(df_['bmi'], bins=[0, 18.5, 25, 30, 40 , float('Inf')], labels=['Underweight', 'Normal', 'Overweight', 'Obese', 'Ultra_obese '])
    df_[df_['bmi']>35]
    df_['senility']= pd.cut(df_['age'], bins=[0, 30, 50, 60,float('Inf')], labels=['Young_adults', 'Adults', 'Old_adults', 'Pensioners'])
    df_['multiplier']= df_['age'] * df_['bmi']
    x=df_.copy().drop('charges', axis=1)
    y=df_['charges'].copy()
    if(scal):
        x = scaler(x)
    return x, y

Model evaluation


- Removed outliers
- No Missing Values
- Power Transform on target

### Feature Engineering:
- 'weight': categorical BMI
- 'senility': categorical Age
- 'multiplier': age * bmi

### Pipeline:
1. Categorical values: JamesSteinEncoder
2. Numerical values: StandardScaler
3. Model

In [13]:
df = pd.read_csv('archive\insurance.csv')
X,y=preprocess(df, scal=False)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.2, random_state= 42)




ParserError: Error tokenizing data. C error: Expected 1 fields in line 9, saw 2


Szukamy optymalnych parametrów poprzez kros - walidacje

In [8]:

def getModelDIc( model, dropCols = [] , powtransform= False):
    

  modelpipe = getPipe(model, dropCols)
  
  
  return {'model': modelpipe, 'powerTransform': powtransform, 'drop': dropCols}



In [ ]:

X_dummy = pd.get_dummies(X_train, drop_first= True)
X_dummy= pd.get_dummies(X_test, drop_first= True)


In [128]:
metrics = pd.DataFrame(create_metrics_tag())

models={}

name='Linear'
model= LinearRegression()
dropCols=[]
models[name]=getModelDIc( model ,dropCols)


name='Tree'
model= DecisionTreeRegressor(max_depth=4)
dropCols=['multiplier', 'region', 'weight', 'senility', 'sex']
models[name]=getModelDIc( model, dropCols)


name='RF RandomSearch'
model= RandomForestRegressor(n_estimators= 1577,
 min_samples_split= 2,
 min_samples_leaf= 4,
 max_features='sqrt',
 max_depth = 10,
 bootstrap= True )
dropCols=[]
models[name]=getModelDIc( model, dropCols, True)



name='RF HyperOpt'
model= RandomForestRegressor(n_estimators = 1089, max_depth=29, min_samples_leaf=12, random_state=1, max_features=7)
dropCols=[]
models[name]=getModelDIc( model, dropCols, False)


name='RF HyperOpt Tran'
model= RandomForestRegressor(n_estimators = 1089, max_depth=29, min_samples_leaf=12, random_state=1, max_features=7)
dropCols=[]
models[name]=getModelDIc( model, dropCols, True)



name='XGBRegressor HyperOpt'
model = XGBRegressor(seed=0, learning_rate=  0.02754445567920848, max_depth= 3, n_estimators= 164)
dropCols=[]
models[name]=getModelDIc( model, dropCols, True)                    


for modelN in models:

    X_train_, X_test_, y_train_, y_test_= X_train, X_test, y_train, y_test
    model=models[modelN]['model']
    if(models[modelN]['powerTransform']):
        y_train_=np.log1p(y_train_)
    if(models[modelN]['drop']):
        X_train_=X_train_.drop(models[modelN]['drop'], axis=1)
        X_test_=X_test_.drop(models[modelN]['drop'], axis=1)
    model.fit(X_train_, y_train_)

    y_pred= model.predict(X_test_)
    y_pred=np.where(y_pred>0, y_pred,0 )
    if(models[modelN]['powerTransform']):
        y_pred=np.expm1(y_pred)
    
    metric=get_metrics(y_test_, y_pred, modelN)
    metrics = pd.concat([metrics, metric]).reset_index(drop=True)


,Specific,MAPE,SMAPE,MSLE,MSE,MAE,R2
0,Linear,0.529010,44.638105,1.485989,3.426320e+07,4317.969150,0.779301
1,Linear transformed full,0.283271,26.256579,0.181731,6.166631e+07,3953.534081,0.602790
2,Linear transformed without,0.284528,26.396397,0.183120,5.920472e+07,3940.640734,0.618646
3,Tree,0.331120,28.100890,0.178438,2.109348e+07,2697.765431,0.864131
4,Random Forest transformed full randomsearch,0.192487,19.846059,0.129524,2.181908e+07,2326.851847,0.859457
5,Random Forest transformed without randomsearch,0.179542,18.452904,0.128546,1.891483e+07,1962.318951,0.878164
6,Random Forest transformed without randomsearc...,0.182685,18.827000,0.126999,1.903440e+07,2063.244743,0.877394
7,RF hyperopted transformed full,0.166282,17.653312,0.122768,1.826791e+07,1939.920439,0.882331
8,RF hyperopted transformed without region&mult...,0.173471,18.223804,0.123695,1.882772e+07,2050.664907,0.878725
9,RF hyperopted transformed without,0.182492,18.823064,0.127118,1.907494e+07,2066.787136,0.877133


In [ ]:
color = (metrics2.MAPE<0.17).map({True: 'background-color: blue', False: ''})
display(metrics2.style.apply(lambda s: color))

In [ ]:
metricsDF= metrics.set_index('Specific')
metricNames=metricsDF.columns
modelNames=metricsDF.index

In [ ]:
fig, axes = plt.subplots(figsize=(11,11),nrows=2, ncols=3)
ax1=plt.subplot(2,3,1)

colors=['b', 'g', 'r', 'c', 'm', 'k']

metricsDF["MAPE"].plot(ax=axes[0,0], kind='bar', grid=True, title= 'MAPE', color=colors[0] )
metricsDF["SMAPE"].plot(ax=axes[0,1], kind='bar', grid=True, title= 'SMAPE' , color=colors[1])
metricsDF["MSLE"].plot(ax=axes[0,2], kind='bar', grid=True, title= 'MSLE', color=colors[2])
metricsDF["MSE"].plot(ax=axes[1,0], kind='bar', grid=True, title ='MSE', color=colors[3])
metricsDF["MAE"].plot(ax=axes[1,1], kind='bar', grid=True, title ='MAE', color=colors[4])
metricsDF["R2"].plot(ax=axes[1,2], kind='bar', grid=True, title ='R2', color=colors[5])


ax1.grid(True)


#Optimalization

In [ ]:
metrics2 = pd.DataFrame(create_metrics_tag())

models={}

name='RF with weight '
model= RandomForestRegressor(n_estimators = 1089, max_depth=29, min_samples_leaf=12, random_state=1, max_features=7)
dropCols=['multiplier', 'senility']
models[name]=getModelDIc( model, dropCols, True)

name='XGBR with added'
model = XGBRegressor(seed=0, learning_rate=  0.02754445567920848, max_depth= 3, n_estimators= 164)
dropCols=[]
models[name]=getModelDIc( model, dropCols, True)   

name='XGBR without added'
model = XGBRegressor(seed=0, learning_rate=  0.02754445567920848, max_depth= 3, n_estimators= 164)
dropCols=['multiplier', 'senility', 'weight']
models[name]=getModelDIc( model, dropCols, True)  

name='XGBR with multiplier'
model = XGBRegressor(seed=0, learning_rate=  0.02754445567920848, max_depth= 3, n_estimators= 164)
dropCols=['weight']
models[name]=getModelDIc( model, dropCols, True)  


name='XGBR with senility'
model = XGBRegressor(seed=0, learning_rate=  0.02754445567920848, max_depth= 3, n_estimators= 164)
dropCols=['multiplier','weight']
models[name]=getModelDIc( model, dropCols, True)  

name='XGBR with multiplier'
model = XGBRegressor(seed=0, learning_rate=  0.02754445567920848, max_depth= 3, n_estimators= 164)
dropCols=['senility', 'weight']
models[name]=getModelDIc( model, dropCols, True)  



for modelN in models:

    X_train_, X_test_, y_train_, y_test_= X_train, X_test, y_train, y_test
    model=models[modelN]['model']
    if(models[modelN]['powerTransform']):
        y_train_=np.log1p(y_train_)
    if(models[modelN]['drop']):
        X_train_=X_train_.drop(models[modelN]['drop'], axis=1)
        X_test_=X_test_.drop(models[modelN]['drop'], axis=1)
    model.fit(X_train_, y_train_)

    y_pred= model.predict(X_test_)
    y_pred=np.where(y_pred>0, y_pred,0 )
    if(models[modelN]['powerTransform']):
        y_pred=np.expm1(y_pred)
    
    metric=get_metrics(y_test_, y_pred, modelN)
    metrics2 = pd.concat([metrics2, metric]).reset_index(drop=True)

In [ ]:
color = (metrics2.MAPE<0.15).map({True: 'background-color: blue', False: ''})
display(metrics2.style.apply(lambda s: color))

In [ ]:
metricsDF2= metrics2.set_index('Specific')
metricNames2=metricsDF2.columns
modelNames2=metricsDF2.index

In [ ]:
fig, axes = plt.subplots(figsize=(10,10),nrows=2, ncols=3)
ax1=plt.subplot(2,3,1)

colors=['b', 'y', 'r', 'c', 'm', 'k']

highlight = 'XGBR without multiplier'
pos = metricsDF2.index.get_loc(highlight)

metricsDF2["MAPE"].plot(ax=axes[0,0], kind='bar', grid=True, title= 'MAPE', color=colors[0] )
metricsDF2["SMAPE"].plot(ax=axes[0,1], kind='bar', grid=True, title= 'SMAPE' , color=colors[1])
metricsDF2["MSLE"].plot(ax=axes[0,2], kind='bar', grid=True, title= 'MSLE', color=colors[2])
metricsDF2["MSE"].plot(ax=axes[1,0], kind='bar', grid=True, title ='MSE', color=colors[3])
metricsDF2["MAE"].plot(ax=axes[1,1], kind='bar', grid=True, title ='MAE', color=colors[4])
metricsDF2["R2"].plot(ax=axes[1,2], kind='bar', grid=True, title ='R2', color=colors[5])


axes[0,0].patches[pos].set_facecolor('g')
axes[0,1].patches[pos].set_facecolor('g')
axes[0,2].patches[pos].set_facecolor('g')
axes[1,0].patches[pos].set_facecolor('g')
axes[1,1].patches[pos].set_facecolor('g')
axes[1,2].patches[pos].set_facecolor('g')

ax1.grid(True)


#Feature Importances

In [ ]:
metrics2 = pd.DataFrame(create_metrics_tag())

models={}

name='XGBR with sensilty'
modelN = XGBRegressor(seed=66, learning_rate=  0.02754445567920848, max_depth= 3, n_estimators= 164)
dropCols=['multiplier','weight' ]
models[name]=getModelDIc( modelN, dropCols, True)  

for modelN in models:

    X_train_, X_test_, y_train_, y_test_= X_train, X_test, y_train, y_test
    model=models[modelN]['model']
    if(models[modelN]['powerTransform']):
        y_train_=np.log1p(y_train_)
    if(models[modelN]['drop']):
        X_train_=X_train_.drop(models[modelN]['drop'], axis=1)
        X_test_=X_test_.drop(models[modelN]['drop'], axis=1)
    model.fit(X_train_, y_train_)
    y_pred= model.predict(X_test_)
    y_pred=np.where(y_pred>0, y_pred,0 )
    if(models[modelN]['powerTransform']):
        y_pred=np.expm1(y_pred)
    
    metric=get_metrics(y_test_, y_pred, modelN)
    metrics2 = pd.concat([metrics2, metric]).reset_index(drop=True)
    importances = model.named_steps["classifier"].feature_importances_


columns= X_train.columns
columns = [i for i in columns if i not in dropCols]

dfimportances=pd.DataFrame(data=importances.reshape(1,9- len(dropCols)), columns= columns)
display(dfimportances)


dfimportances.plot.bar()


In [ ]:

models={}

name='XGBR'
modelN = XGBRegressor(seed=66, learning_rate=  0.02754445567920848, max_depth= 3, n_estimators= 164)
dropCols=['multiplier','weight' ,'senility']
models[name]=getModelDIc( modelN, dropCols, True)  

for modelN in models:

    X_train_, X_test_, y_train_, y_test_= X_train, X_test, y_train, y_test
    model=models[modelN]['model']
    if(models[modelN]['powerTransform']):
        y_train_=np.log1p(y_train_)
    if(models[modelN]['drop']):
        X_train_=X_train_.drop(models[modelN]['drop'], axis=1)
        X_test_=X_test_.drop(models[modelN]['drop'], axis=1)
    model.fit(X_train_, y_train_)
    y_pred= model.predict(X_test_)
    y_pred=np.where(y_pred>0, y_pred,0 )
    if(models[modelN]['powerTransform']):
        y_pred=np.expm1(y_pred)


## Creating predictions

In [ ]:


columns= X_train.columns
columns = [i for i in columns if i not in dropCols]

Trials= pd.DataFrame(columns=columns )




In [ ]:
lists=[]
lists.append( [24, 'male', 23, 1, 'yes', 'southeast'])
lists.append([35, 'female', 33, 2, 'no', 'northwest'])
lists.append( [59, 'female', 25, 1, 'no', 'southeast'])
lists.append([63, 'male', 30, 2, 'yes', 'northwest'])

for list in lists:
    Trials= Trials.append(pd.DataFrame([list], columns= columns), ignore_index=True)

y_pred= model.predict(Trials)
y_pred=np.where(y_pred>0, y_pred,0 )
y_pred=np.expm1(y_pred)

Trials['charges']= y_pred
Trials



#RandomSearchGrid

In [138]:

space_rf = {"max_depth": hp.randint('max_depth', 1, 100),
             "n_estimators": scope.int(hp.qloguniform('n_estimators', 1, 10, 1)),
             "min_samples_leaf": hp.randint('min_samples_leaf', 1, 50),
             "max_features": hp.choice('max_features', ['sqrt', 2, 3, 4, 5, 6, 7, 8]),
             }
def minimize_rf(params):
    max_depth = params["max_depth"]
    max_features = params['max_features']
    min_samples = params['min_samples_leaf']
    n_estimators = int(params['n_estimators'])
    model = getPipe(RandomForestRegressor(max_depth=max_depth, max_features=max_features, min_samples_leaf=min_samples, n_estimators=n_estimators))
    mean_cv_score = np.mean(cross_val_score(model, X_train, y_train))
    return -mean_cv_score


In [101]:
best_rf = fmin(minimize_rf, space_rf, max_evals=20, algo=tpe.suggest)
best_rf

100%|██████████| 20/20 [05:55<00:00, 17.79s/trial, best loss: -0.8492072062836253]


{'max_depth': 47,
 'max_features': 5,
 'min_samples_leaf': 12,
 'n_estimators': 2639.0}

In [10]:
#best_rf_regressor = RandomForestRegressor(n_estimators = 365, max_depth=47, min_samples_leaf=6, random_state=1)
{'max_depth': 47,
 'max_features': 5,
 'min_samples_leaf': 12,
 'n_estimators': 2639.0}

model = RandomForestRegressor(n_estimators = 2639, max_depth=47, min_samples_leaf=12, random_state=1, max_features=5)
best_rf_regressor = getPipe(model)
best_rf_regressor.fit(X_train, y_train)
y_pred_best = best_rf_regressor.predict(X_test)
r2_score(y_test, y_pred_best)

0.8792472964158641

In [11]:
importances = best_rf_regressor.named_steps["classifier"].feature_importances_


pd.DataFrame(data=importances.reshape(1,9), columns= X_train.columns)

,age,sex,bmi,children,smoker,region,weight,senility,multiplier
0,0.079161,0.074294,0.008274,0.062669,0.002005,0.695223,0.003543,0.060741,0.014091


#Hyperopt XGBoost

In [97]:


space={'max_depth': hp.quniform("max_depth", 3, 18, 1),
        'gamma': hp.uniform ('gamma', 1,9),
        'reg_alpha' : hp.quniform('reg_alpha', 40,180,1),
        'reg_lambda' : hp.uniform('reg_lambda', 0,1),
        'colsample_bytree' : hp.uniform('colsample_bytree', 0.5,1),
        'min_child_weight' : hp.quniform('min_child_weight', 0, 10, 1),
        'n_estimators': hp.quniform("n_estimators", 3, 18, 1),
    }

In [124]:
space = {
    'learning_rate': hp.uniform('learning_rate', 0.001, 0.3),
    'max_depth': scope.int(hp.quniform("max_depth", 3, 18, 1)),
    'gamma': scope.int(hp.uniform ('gamma', 1,9)),
    'reg_alpha' : scope.int(hp.quniform('reg_alpha', 40,180,1)),
    'reg_lambda' : hp.uniform('reg_lambda', 0,1),
    'colsample_bytree' : hp.uniform('colsample_bytree', 0.5,1),
     'min_child_weight' : scope.int(hp.quniform('min_child_weight', 0, 10, 1)),
    'n_estimators': scope.int(hp.quniform("n_estimators", 3, 18, 1))
}




In [125]:
def objective(params):
    
    xgboost = XGBRegressor(seed=0, **params)
    score = cross_val_score(estimator=xgboost, 
                            X=X_trains, 
                            y=y_train, 
                            cv=3, 
                            n_jobs=-1).mean()
    # Loss is negative score
    loss = - score
    # Dictionary with information for evaluation
    return {'loss': loss, 'params': params, 'status': STATUS_OK}

best = fmin(fn = objective, space = space, algo = tpe.suggest, max_evals = 60, trials = Trials())


100%|██████████| 60/60 [00:02<00:00, 22.98trial/s, best loss: -0.8492629086004975]


In [126]:
print(space_eval(space, best))


{'colsample_bytree': 0.9108343039685818, 'gamma': 7, 'learning_rate': 0.23825571365013604, 'max_depth': 3, 'min_child_weight': 5, 'n_estimators': 16, 'reg_alpha': 94, 'reg_lambda': 0.4490642002880383}


In [110]:
xgboost_bo = XGBRegressor(seed=0, 
                           learning_rate=space_eval(space, best)['learning_rate'], 
                           max_depth=space_eval(space, best)['max_depth'], 
                           n_estimators=space_eval(space, best)['n_estimators'],
                           ).fit(X_trains,y_train)
# Make prediction using the best model
bayesian_opt_predict = xgboost_bo.predict(X_tests)
# Get performance metrics
f1s = mean_squared_error(y_test, bayesian_opt_predict)
# Print result
print(f'The f1  value for the xgboost Bayesian optimization is {f1s:.4f}')

xgboost_bo.score(X_tests, y_test)

The f1  value for the xgboost Bayesian optimization is 18574143.1462


0.8803588089401068